# DLC Live PyTorch Demo

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dlclive import DLCLive
import cv2
import numpy as np
from pathlib import Path
import time

### Snapshot to ONNX model 

In [ ]:
# In case you do not have a .onnx model exported, use this cell to export your DLC3.0 snapshot

from deeplabcut.pose_estimation_pytorch.config import read_config_as_dict
from deeplabcut.pose_estimation_pytorch.models import PoseModel
import torch
import onnxruntime as ort

device = "cuda" if torch.cuda.is_available() else "cpu"

#Dikra
# root = Path("/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy")
# model_cfg = read_config_as_dict(root / "pytorch_config.yaml")
# weights_path = root / "snapshot-200.pt"

#Anna
root = Path("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train")
model_cfg = read_config_as_dict(root / "pytorch_config.yaml")
weights_path = root / "snapshot-263.pt"

model = PoseModel.build(model_cfg["model"])
weights = torch.load(weights_path, map_location=device)
model.load_state_dict(weights["model"])

dummy_input = torch.zeros((1, 3, 224, 224))

torch.onnx.export(
    model,
    dummy_input,
    "/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/resnet.onnx",
    verbose=False,
    input_names=["input"],
    dynamic_axes={"input": {0: "batch_size", 2: "height", 3: "width"}},
)

In [ ]:
# Load test frame
img = cv2.imread("/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/img008.png")

### DLC Live with ONNX exported DLC 3.0 model

In [ ]:
# Dikra
onnx_dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    model_type="onnx",
    device="cuda",
    display=True,
)

# Anna
# onnx_dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt')
# onnx_dlc_live = DLCLive("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/DLC_dev-single-animal_resnet_50_iteration-1_shuffle-1", processor=dlc_proc)
# img = cv2.imread("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/img049.png")

onnx_pose = onnx_dlc_live.init_inference(frame=img)
onnx_pose

![Screenshot from 2024-08-20 14-29-53.png](./docs/assets/Screenshot%20from%202024-08-20%2014-36-00.png)

In [ ]:
onnx_pose = onnx_dlc_live.get_pose(frame=img)
onnx_pose

In [ ]:
onnx_dlc_live.display.destroy()

### DLC Live with snaptshot of DLC 3.0 model (.pt)

In [ ]:
# Dikra
pytorch_dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    snapshot="snapshot-200.pt",
    device="cuda",
    model_type="pytorch",
    display=True,
)

# Anna
# pytorch_dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt')
# pytorch_dlc_live = DLCLive("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/DLC_dev-single-animal_resnet_50_iteration-1_shuffle-1", processor=dlc_proc)
# img = cv2.imread("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/img049.png")

pytorch_pose = pytorch_dlc_live.init_inference(frame=img)
pytorch_pose

In [ ]:
pytorch_dlc_live.display.destroy()

![PyTorch model inference](./docs/assets/Screenshot%20from%202024-08-20%2014-29-53.png)

### Which is faster?

In [ ]:
import glob
import os

root = "/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy"
test_images = glob.glob(os.path.normpath(root + "/*.png"))


def mean_time_inference(dlc_live, images):
    times = []
    for i, img_p in enumerate(images):
        img = cv2.imread(img_p)

        if i == 0:
            start = time.time()
            dlc_live.init_inference(img)
            end = time.time()
        else:
            start = time.time()
            dlc_live.get_pose(img)
            end = time.time()
        times.append(end - start)
    print(times)

    return np.mean(times), times

In [ ]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    device="cuda",
    model_type="onnx",
    display=True,
)

mean_time = mean_time_inference(dlc_live, test_images)
print(
    f"TOTAL Inference of ONNX model took on average {mean_time} seconds for {len(test_images)} images"
)

In [ ]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    snapshot="snapshot-200.pt",
    device="cuda",
    model_type="pytorch"
)
root = "/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy"
test_images = glob.glob(os.path.normpath(root + "/*.png"))

mean_time = mean_time_inference(dlc_live, test_images) 
print(f"Inference of PyTorch model took on average {mean_time} seconds for {len(test_images)} images")

# Benchmarking

Currently the benchmark_pytorch.py script serves to provide a function for analyzing a preexisting video to test PyTorch for running video inference in DLC-Live. Code for running video inference on a live video feed is WIP.

For true benchmarking purposes, we aim to add feature for recording the time it takes to analyze each frame / how many frames can be analyzed per second. Discuss what measure to use and consult the DLC Live paper

In [ ]:
# New version with DLCLive object included in the code

# Import the analyze_video function from the file where it's defined
from dlclive.benchmark_pytorch import analyze_video

# Define the paths
video_path = '/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/1_20cms_0degUP_first_03s.avi'
model_path = '/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train'

import cProfile
import io
import pstats

pr = cProfile.Profile()
pr.enable()

# Call the analyze_video function with the appropriate arguments
poses = analyze_video(
    video_path=video_path,
    model_path=model_path,
    model_type="onnx",  # Assuming you're using an ONNX model
    device="cuda",      # Use "cpu" if not using CUDA
    display=False,       # Set to True if you want to display the video while processing
    save_poses=True,
    save_dir='output_directory',  # Directory to save the output files
    draw_keypoint_names=True      # Set to True to draw keypoint names on the video
)

#'poses' will contain the list of poses detected

# Create a stream to capture the profiler's output
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()

# Print the profiling output
print(s.getvalue())

In [ ]:
# test download of benchmarking dataset 
#OBS link it not working, waiting for updated link to benchmarking dataset

# from dlclive.benchmark_pytorch import download_benchmarking_data

# download_benchmarking_data()



In [1]:
from dlclive.benchmark_pytorch import get_system_info
import platform

sys_info = get_system_info()

Loading DLC 3.0.0rc2...


/Users/annastuckert/anaconda3/envs/deeplabcut3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/annastuckert/anaconda3/envs/deeplabcut3/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
print(sys_info)

{'host_name': 'MCKDQTN4YXDV', 'op_sys': 'macOS-14.5-arm64-arm-64bit', 'python': 'deeplabcut3', 'device_type': 'CPU', 'device': ['Apple M2'], 'freeze': ['absl-py==2.1.0', 'alabaster==0.7.16', 'albumentations==1.4.3', 'app-model==0.2.7', 'appdirs==1.4.4', 'appnope==0.1.4', 'asttokens==2.4.1', 'astunparse==1.6.3', 'attrs==23.2.0', 'Babel==2.15.0', 'blosc2==2.0.0', 'build==1.2.1', 'cachetools==5.4.0', 'cachey==0.2.1', 'certifi==2024.7.4', 'charset-normalizer==3.3.2', 'click==8.1.7', 'cloudpickle==3.0.0', 'colorcet==3.1.0', 'coloredlogs==15.0.1', 'comm==0.2.2', 'contourpy==1.2.1', 'cycler==0.12.1', 'Cython==3.0.10', 'dask==2024.7.0', 'dask-expr==1.1.7', 'dask-image==2024.5.3', 'debugpy==1.8.2', 'decorator==5.1.1', 'deeplabcut @ git+https://github.com/DeepLabCut/DeepLabCut.git@83f1acb9c179caf46353cdb211318b29832ff6ec', 'deeplabcut-live==1.0.2', 'deeplabcut-live-gui @ file:///Users/annastuckert/Documents/DeepLabCut-live-GUI', 'dill==0.3.8', 'dlclibrary==0.0.6', 'docker-pycreds==0.4.0', 'docst